In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
import os
import pandas as pd
import csv
import json
import re


In [ ]:
# File path of the JSON file
file_path1 = 'drive/My Drive/Projects/LSE-Symptomatology/Data/PrimaryLabeled_Dataset(Post)'

In [ ]:
file_path2 = 'drive/My Drive/Projects/LSE-Symptomatology/Data/PrimaryLabeled_Dataset(Comment)'

In [ ]:
# Load the JSON data from the file
data_submission = []

with open(file_path1, 'r') as file:
    for line in file:
        data_submission.append(json.loads(line))

In [ ]:
# Convert the list of dictionaries to a DataFrame
df_submission = pd.DataFrame(data_submission)

In [ ]:
df_submission.head()

,postID,authorID,date,flairName,title,post,num_comments,score,upvote,downvote
0,cx393,[deleted],2010-08-04 02:25:45,None,Question about weight loss and PCOS.,"I'm 26, and can't afford health insurance for ...",6,8,NaN,NaN
1,cxl3p,improbablywrong,2010-08-05 05:16:03,None,Progesterone cream - do you use it?,I think this is a great treatment for PCOS. Id...,1,5,NaN,NaN
2,dd89k,hgielrehtaeh,2010-09-13 15:16:16,None,"Really, really, REALLY struggling with my inab...",I've had 7 cousins have babies in the last two...,8,7,NaN,NaN
3,ds0n2,cisforcourtney,2010-10-16 11:19:46,None,Question About Clothing,I recently gained some weight and it seems lik...,9,7,NaN,NaN
4,dylgr,GuiltEngine,2010-10-30 05:40:06,None,PCOS and hair loss,"Hallo there, very specialized subreddit that I...",0,1,NaN,NaN


In [ ]:
len(df_submission)

59962

In [ ]:
# Load the JSON data from the file
data_comments = []

with open(file_path2, 'r') as file:
    for line in file:
        data_comments.append(json.loads(line))

In [ ]:
df_comments = pd.DataFrame(data_comments)

In [ ]:
df_comments.head()

,commentID,authorID,date,comment,parentID,linkID
0,c0uubmc,improbablywrong,2010-07-21 22:15:28,"I've tried several items on this list, like Me...",t3_cs80y,t3_cs80y
1,c0ux9q2,emmster,2010-07-22 20:41:36,"I eat a generally lower carb diet (That's ""die...",t3_cs80y,t3_cs80y
2,c0uxcw1,vixiera,2010-07-22 21:13:36,"I self-diagnosed myself a few years ago, but I...",t3_cs80y,t3_cs80y
3,c0uxxpr,improbablywrong,2010-07-23 00:55:28,"I haven't visited their forum for a while, but...",t3_csm11,t3_csm11
4,c0uy2d1,Amberkins72187,2010-07-23 01:48:48,I have had that site bookmarked forever. I hav...,t3_csm11,t3_csm11


In [ ]:
len(df_comments)

641441

In [ ]:
# Define the symptoms and their variations
symptoms ={'irregular periods': ['irregular periods',
  'irregular bleeding',
  'irregular menstrual cycles',
  'irregular cycles',
  'irregular period',
  'abnormal periods',
  'inconsistent periods',
  'frequent periods',
  'period irregularity',
  'very irregular periods',
  'extremely irregular periods',
  'unpredictable periods',
  'regular periods',
  'irregular menstrual cycle',
  'irregular menstruation',
  'irregular menses',
  'irregular cycle'],
 'no period': ['absent periods',
  'absence of menstruation',
  'missed period',
  'missing periods',
  'missing period',
  'absence of period',
  'lack of periods',
  'infrequent periods',
  'no period',
  'amenorrhea',
  'absence of periods',
  'missed periods',
  'late period',
  'no periods'],
 'heavy periods': ['heavy periods',
  'heavy bleeding',
  'heavy flow',
  'long periods',
  'heavy period',
  'prolonged periods',
  'prolonged period',
  'prolonged bleeding'],
 'blood clots': ['blood clots'],
 'brown discharge': ['brown discharge', 'brown spotting'],
 'spotting': ['spotting', 'spots'],
 'painful periods': ['painful periods',
  'period pain',
  'painful period',
  'terrible periods',
  'pain during periods',
  'horrible periods'],
 'night sweats': ['night sweats', 'sweating', 'hot flashes'],
 'hirsutism': ['hirsutism',
  'hirtuism',
  'hirituism',
  'hirutism',
  'hirsuitism',
  'excess body hair',
  'excess hair',
  'excess hair growth',
  'abnormal hair growth',
  'unwanted hair growth',
  'excessive hair growth',
  'facial hair',
  'hair on face',
  'body hair',
  'hair growth',
  'facial hair growth',
  'chin hair',
  'chin hairs'],
 'hair loss': ['hair loss',
  'hair thinning',
  'balding',
  'baldness',
  'hair falling out',
  'hair fall',
  'thinning hair',
  'receding hairline',
  'thin hair'],
 'high androgen levels': ['high androgen levels',
  'high androgens',
  'elevated androgens',
  'high androstenedione'],
 'high testosterone': ['high testosterone',
  'high levels of testosterone',
  'elevated testosterone'],
 'high dheas': ['high dheas',
  'high dhea-s',
  'high dhea-s',
  'high dhea',
  'elevated dhea-s',
  'dheas'],
 'hormonal imbalance': ['hormonal imbalance',
  'hormone imbalance',
  'hormonal imbalances',
  'hormone imbalances'],
 'high estrogen': ['high estrogen', 'estrogen dominance'],
 'low estrogen': ['low estrogen'],
 'high lh levels': ['high lh levels'],
 'high prolactin': ['high prolactin'],
 'weight gain': ['weight gain',
  'difficulty losing weight',
  'obese',
  'obesity',
  'overweight',
  'weight issues',
  'trouble losing weight',
  'weight fluctuations',
  'inability to lose weight',
  'weight problems',
  'weight fluctuation',
  'weight struggles',
  'weight management'],
 'belly fat': ['belly fat'],
 'weight loss': ['weight loss','loss weight'],
 'hunger/loss of appetite': ['hunger',
  'loss of appetite',
  'increased appetite',
  'no appetite'],
 'frequent urination': ['frequent urination', 'increased urination'],
 'inflammation': ['inflammation'],
 'hypothyroidism': ['hypothyroidism',
  'thyroid issues',
  'hypothyroid',
  'thyroid'],
 'diabetes': ['diabetes',
  'low blood sugar',
  'pre-diabetic',
  'hypoglycemia',
  'prediabetes',
  'prediabetic',
  'pre-diabetes'],
 'insulin resistance': ['insulin resistance',
  'insulin resistant',
  'ir (insulin resistance)'],
 'high cholesterol': ['high cholesterol',
  'high triglycerides',
  'high ldl cholesterol'],
 'high insulin levels': ['high insulin levels',
  'high glucose',
  'high insulin',
  'high glucose'],
 'cysts on ovaries': ['cysts on ovaries',
  'enlarged ovary',
  'polycystic ovary',
  'enlarged ovaries',
  'cyst',
  'cysts',
  'follicles',
  'ovarian cysts',
  'polycystic ovaries',
  'cysts in ovaries',
  'cystic ovaries',
  'ovarian cyst',
  'cyst on ovary'],
 'endometriosis': ['endometriosis', 'endometrioma'],
 'cramps': ['cramps', 'cramping'],
 'pelvic pain': ['pelvic pain'],
 'ovary pain': ['ovary pain', 'ovarian pain', 'pain in ovaries'],
 'back pain': ['back pain', 'leg pain'],
 'abdominal pain': ['abdominal pain', 'pain in lower abdomen'],
 'low libido': ['low libido', 'low sex drive', 'decreased libido'],
 'infertility': ['infertility',
  'fertility issues',
  'infertility problems',
  'fertility'],
 'ovulation/annovulation': ['ovulation', 'anovulation', 'ovulation issues'],
 'pain during sex': ['pain during sex',
  'painful sex',
  'pain during intercourse'],
 'acne': ['acne', 'breakouts', 'pimples'],
 'dry skin': ['dry skin'],
 'redness': ['redness', 'red marks', 'red spots'],
 'skin tags': ['skin tags'],
 'stretch marks': ['stretch marks'],
 'skin issues': ['skin issues',
  'hyperpigmentation',
  'itching',
  'scars',
  'eczema',
  'rosacea',
  'sensitive skin'],
 'acanthosis nigricans': ['acanthosis nigricans', 'dark spots'],
 'oily skin': ['oily skin'],
 'mood swings': ['mood swings', 'anxiety', 'depression', 'feeling sick'],
 'insomnia': ['insomnia',
  'sleep apnea',
  'trouble sleeping',
  'sleep disturbances',
  'poor sleep',
  'sleep issues',
  'snoring'],
 'brain fog': ['brain fog', 'foggy head'],
 'nausea': ['nausea', 'nauseated', 'nauseous'],
 'bloating': ['bloating', 'bloated', 'bloat'],
 'vomiting': ['vomiting', 'vomit'],
 'constipation': ['constipation'],
 'stomach pain': ['stomach pain', 'stomach issues'],
 'digestive issues': ['digestive issues',
  'gut issues',
  'indigestion',
  'gluten intolerance'],
 'anemia': ['anemia', 'low iron', 'iron deficiency', 'low ferritin'],
 'pain': ['pain', 'discomfort', 'pains'],
 'dizziness': ['dizziness', 'lightheadedness', 'dizzy', 'fainting'],
 'headaches': ['headaches', 'migraines', 'headache', 'migraine'],
 'fatigue': ['fatigue',
  'tiredness',
  'weakness',
  'low energy',
  'exhaustion',
  'fatigued'],
 'cravings': ['cravings', 'sugar cravings'],
 'breast tenderness': ['breast tenderness',
  'sore breasts',
  'breast pain',
  'sore nipples']}


In [ ]:
# Initialize a dictionary to store the count of unique users per symptom
symptom_user_count = {symptom: set() for symptom in symptoms}

In [ ]:
# Function to search for symptom mentions
def search_symptoms(text, symptom_variations):
    for variation in symptom_variations:
        if re.search(r'\b' + re.escape(variation.rstrip('s')) + r's?\b', text, re.IGNORECASE):
            return True
    return False

In [ ]:
for index, row in df_submission.iterrows():
    # Concatenate 'title' and 'post'
    combined_text = (row.get('title', '') + " " + row.get('post', '')).strip()

    for symptom, variations in symptoms.items():
        if search_symptoms(combined_text, variations):
            symptom_user_count[symptom].add(row['authorID'])


In [ ]:
for index, row in df_comments.iterrows():
    for symptom, variations in symptoms.items():
        if search_symptoms(row['comment'], variations):
            symptom_user_count[symptom].add(row['authorID'])



In [ ]:
# Convert the sets to counts
symptom_user_count = {symptom: len(usernames) for symptom, usernames in symptom_user_count.items()}

# Convert to DataFrame for better visualization
symptom_count_df = pd.DataFrame(symptom_user_count.items(), columns=['Symptom', 'Unique User Count'])




In [ ]:
# Sort the DataFrame by 'Unique User Count' in descending order
symptom_sort_count_df = symptom_count_df.sort_values(by='Unique User Count', ascending=False)

# Reset the index for clean formatting
symptom_sort_count_df.reset_index(drop=True, inplace=True)

# Display the results
print(symptom_sort_count_df)

               Symptom  Unique User Count
0            hirsutism              16001
1                 acne              14256
2     cysts on ovaries              13781
3                 pain              13495
4          weight gain              13150
..                 ...                ...
59     pain during sex                261
60  frequent urination                160
61             redness                108
62         weight loss                 45
63      high lh levels                 31

[64 rows x 2 columns]


In [ ]:
# Save the results to a CSV file
output_file_path = 'drive/My Drive/Data/Data Analysis Outcomes/Unique_user_symptoms_Sortedfrequency(PCOS_subreddit_final).csv'  # Update this with desired file path
symptom_sort_count_df.to_csv(output_file_path, index=False)

